In [24]:
from IPython.display import clear_output
import time
try:
    
    !pip install h5py
    !pip install pandas
    !pip install Pillow
    !pip install matplotlib
    #!pip install sklearn
    !pip install scikit-learn
    !pip install torch
    !pip install torchvision
    time.sleep(5)
    clear_output()
    print("Packages successfully installed")
    
except:
    print("error installing packages")



Packages successfully installed


In [29]:
import h5py
import pandas as pd
import numpy as np
from PIL import Image
import io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models


In [26]:

# Define the custom dataset class
class ISIC2024Dataset(Dataset):
    def __init__(self, metadata: pd.DataFrame, file_hdf: str, transform=None):
        self.metadata = metadata
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, item):
        isic_row = self.metadata.iloc[item]
        isic_id = isic_row.isic_id
        image = np.array(Image.open(io.BytesIO(self.fp_hdf[isic_id][()])))
        label = isic_row.target

        if self.transform:
            image = self.transform(image)

        return image, label



In [7]:
# Load metadata
metadata = pd.read_csv('isic-2024-challenge/train-metadata.csv')
metadata = metadata[['isic_id', 'target']]

# Ensure an even split of target=0 and target=1
target_0 = metadata[metadata['target'] == 0].sample(n=700, random_state=42)
target_1 = metadata[metadata['target'] == 1].sample(n=393, random_state=42)

balanced_metadata = pd.concat([target_0, target_1]).reset_index(drop=True)

# Split the metadata into training and validation sets
train_metadata, val_metadata = train_test_split(balanced_metadata, test_size=0.2, random_state=42)


C:\Users\ritvi\AppData\Local\Temp\ipykernel_24832\435405121.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('isic-2024-challenge/train-metadata.csv')


In [8]:
# Define image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset objects
train_dataset = ISIC2024Dataset(train_metadata, 'isic-2024-challenge/train-image.hdf5', transform=transform)
val_dataset = ISIC2024Dataset(val_metadata, 'isic-2024-challenge/train-image.hdf5', transform=transform)



In [9]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)



In [10]:
# Define the model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Binary classification

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


C:\Users\ritvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ritvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float()
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        # Calculate accuracy
        predicted = (torch.sigmoid(outputs.squeeze()) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')


Epoch [1/10], Loss: 0.5712, Accuracy: 0.7563
Epoch [2/10], Loss: 0.4276, Accuracy: 0.8101
Epoch [3/10], Loss: 0.4085, Accuracy: 0.8146
Epoch [4/10], Loss: 0.3406, Accuracy: 0.8616
Epoch [5/10], Loss: 0.2776, Accuracy: 0.8867
Epoch [6/10], Loss: 0.2457, Accuracy: 0.9039
Epoch [7/10], Loss: 0.2075, Accuracy: 0.9165
Epoch [8/10], Loss: 0.2524, Accuracy: 0.8867
Epoch [9/10], Loss: 0.1674, Accuracy: 0.9325
Epoch [10/10], Loss: 0.2311, Accuracy: 0.9050


In [12]:
# Evaluation loop
model.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device).float()
        
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        
        val_loss += loss.item() * images.size(0)
        
        predicted = (torch.sigmoid(outputs.squeeze()) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_loss = val_loss / len(val_loader.dataset)
accuracy = correct / total
print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')


Validation Loss: 1.5664, Accuracy: 0.7443


In [13]:
metadata['target'].value_counts()

target
0    400666
1       393
Name: count, dtype: int64